<a href="https://colab.research.google.com/github/Nachiket1904/RAG_Chatbot/blob/main/RAG_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
%pip install chromadb
%pip install langchain
%pip install sentence-transformers
%pip install openai

In [ ]:
# Importing data.
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

In [ ]:
# Loading embeddings from ChromaDB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Loading Data
loader = CSVLoader(file_path='/content/drive/MyDrive/Colab Notebooks/raw_data/bhagavad-gita_2.csv', encoding="utf-8")
documents = loader.load()
# Debugging: Print the number of documents loaded
print(f"Number of documents loaded: {len(documents)}")


Number of documents loaded: 700


In [ ]:
# prompt: i have following dataset csv file and i want only following column from that not all so can u write a code for that using pandas library and store that excel file it

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/raw_data/bhagavad-gita.csv')
df
df2 = df[['Title','Chapter', 'Verse', 'Enlgish Translation']]
df2.to_csv('/content/drive/MyDrive/Colab Notebooks/raw_data/bhagavad-gita_2.csv', index=False)


In [ ]:

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(f"Number of chunks: {len(docs)}")

Number of chunks: 700


In [ ]:
# save to disk
db2 = Chroma.from_documents(docs, embeddings, persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db")

In [ ]:
# loading chroma data from disk
db = Chroma(persist_directory="/content/drive/MyDrive/chroma_db", embedding_function=embeddings)

In [ ]:
# Querying data from ChromaDB
query = "What happens after death ?"
docs = db2.similarity_search(query)
docs[0]

Document(page_content='Title: Sankhya Yoga\nChapter: Chapter 2\nVerse: Verse 2.20\nEnlgish Translation: The Blessed Lord said: Dear ARJUNA, the ATMAN or Soul can neither be born nor can it die. It is forever immortal, eternal and ancient. The Soul in a body does not die when the body itself perishes and ceases to exist. The Soul always lives on.', metadata={'row': 65, 'source': '/content/drive/MyDrive/Colab Notebooks/raw_data/bhagavad-gita_2.csv'})

In [ ]:
# Making a string of related verses.
verses = ""

for doc in docs:
    verses += doc.page_content + "\n"

In [ ]:
# Template for prompt.
template = """
Given the following extracted parts of a long document ("SOURCES") and a question ("QUESTION").
Don't try to make up an answer and use the text in the SOURCES only for the answer.
If you don't know the answer, just say that you don't know.
List down all the SOURCES in the response.

QUESTION: {question}
=========
SOURCES:
{verses}
=========
ANSWER:
"""

prompt = PromptTemplate(template=template, input_variables=["question", "verses"])

In [ ]:
# Function to trim tokens in a string.
def trim_string_by_tokens(input_string, num_tokens):
    # Split the string into tokens
    tokens = input_string.split()

    # Ensure the number of tokens to keep is not greater than the total number of tokens
    num_tokens = min(num_tokens, len(tokens))

    # Join the desired number of tokens back together
    trimmed_string = ' '.join(tokens[:num_tokens])

    return trimmed_string

In [ ]:
# Trimming final prompt to fit in context window.
final_prompt = prompt.format(question=query, verses=verses)
final_prompt = (final_prompt, 1500)[0]
final_prompt

'\nGiven the following extracted parts of a long document ("SOURCES") and a question ("QUESTION").\nDon\'t try to make up an answer and use the text in the SOURCES only for the answer.\nIf you don\'t know the answer, just say that you don\'t know.\nList down all the SOURCES in the response.\n\nQUESTION: What happens after death ?\n=========\nSOURCES:\nTitle: Sankhya Yoga\nChapter: Chapter 2\nVerse: Verse 2.20\nEnlgish Translation: The Blessed Lord said: Dear ARJUNA, the ATMAN or Soul can neither be born nor can it die. It is forever immortal, eternal and ancient. The Soul in a body does not die when the body itself perishes and ceases to exist. The Soul always lives on.\nTitle: Sankhya Yoga\nChapter: Chapter 2\nVerse: Verse 2.30\nEnlgish Translation: ARJUNA, although the body can be slain, the Soul cannot. The Soul of a being lives on forever, therefor it is not necessary to grieve over anybody’s death because the most important part of them never dies at all, and that is, their Souls.

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY'


model_name = 'text-davinci-003'

# Generating Response from the selected model
response = openai.ChatCompletion.create(
  model=model_name,
  messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": final_prompt}
  ],
  max_tokens=1500
)

# Display the response
print(response['choices'][0]['message']['content'])
